# Project 3
Members:
 - Andreas Bjelland Berg

In [20]:
from typing import Callable

import matplotlib.pyplot as plt
import numpy as np

## Problem 1
*In this problem, we want to approximate an integral*
$$
    I(f) = \int_0^1 f(x) dx
$$
*by Romberg's method $T_{m, 2}$ with number of function evaluations being $2^{m + 1} + 1$ for natural number $m$*

### Problem 1.1
*Implement your own code which calculates quadrature values using Romberg's method. For a function $f_1(x) = e^{-x}$, at least how many numbers of points do you need to achieve the absolute error $|I(f_1) - T_{m, 2}|$ smaller than $10^{-8}$?*

First, observe that by simple mathematics, we get
$$
    I(f_1) = \int_0^1 e^{-x} dx = 1 - \frac{1}{e}
$$

Observe also that $f_1$ is infinitely derivable, and is continuous on $[0, 1]$.

In [21]:
integral_f1 = 1 - 1 / np.exp(1)

In [22]:
def f1(x: float) -> float:
    return np.exp(-x)

In [29]:
def trapezoid(f: Callable, m: int, a: int, b: int):
    """Estimate the integral of function f using the Trapezoidal method

    Args:
        f (Callable): Function to apply Trapezoidal method to
        m (int): Number of equally spaced points
        a (int): Start of domain
        b (int): End of domain

    Returns:
        _type_: _description_
    """
    if m == 0:
        delta = (b - a)
    else:
        delta = (b - a) / m
    x_list = np.linspace(a, b, num=m+1)

    sum_ = 0
    for x in x_list[1: -1]:
        sum_ += f(x)
    sum_ += (f(x_list[0]) + f(x_list[-1])) / 2
    
    return delta * sum_

def rombergs(f: Callable, m: int, k: int = 2, a: int = 0, b: int = 1):
    """Apply Romberg's method T_{m, n} on function f

    Args:
        f (Callable): Function to apply Romberg's to
        m (int): Base for number of function evaluations. Evaluations are 
            2^{m + 1} + 1
        k (int, optional): Number of recursive steps to apply. Defaults to 2.
        a (int, optional): Start of domain. Defaults to 0
        b (int, optional): End of domain. Defaults to 1
    """
    if k == 0:
        return trapezoid(f, m, a, b)
    
    return (
        (4**k
            * rombergs(f, 2*m, k - 1, a, b)
            - rombergs(f, m, k - 1))
        / (4**k - 1)
    )

In [30]:
treshold = 1e-8

m = 0
while True:
    romberg_estimate = rombergs(f1, m, k=2, a=0, b=1)

    error = np.abs(integral_f1 - romberg_estimate)

    print(f"Error for m={m} ({2**(m+1)+1} evaluations): {error}")
    if error < treshold:
        print(f"Error passed treshold {treshold} at m={m} ({2**(m+1)+1} evaluations)")
        break

    m += 1

Error for m=0 (3 evaluations): 0.36787944117144233
Error for m=1 (5 evaluations): 3.1617976581355123e-07
Error for m=2 (9 evaluations): 5.061798757921565e-09
Error passed treshold 1e-08 at m=2 (9 evaluations)


### Problem 1.2
*Consider another integrand function $f_2(x) = e^{-|x - 1/2|}$. At least how many numbers of points do you need to achieve the absolute error smaller than $10^{-4}$?*

Once again, observe first that
$$
    I(f_2) = \int_0^1 e^{-|x - 1/2|} dx = \frac{-2}{\sqrt{e}} + 2
$$

In [31]:
integral_f2 = -2 / np.sqrt(np.exp(1)) + 2

In [33]:
def f2(x: float) -> float:
    return np.exp(- np.abs(x - 0.5))

In [34]:
treshold = 1e-4

m = 0
while True:
    romberg_estimate = rombergs(f2, m, k=2, a=0, b=1)

    error = np.abs(integral_f2 - romberg_estimate)

    print(f"Error for m={m} ({2**(m+1)+1} evaluations): {error}")
    if error < treshold:
        print(f"Error passed treshold {treshold} at m={m} ({2**(m+1)+1} evaluations)")
        break

    m += 1

Error for m=0 (3 evaluations): 0.18040802086209973
Error for m=1 (5 evaluations): 0.005442243324213392
Error for m=2 (9 evaluations): 6.301527410990104e-09
Error passed treshold 0.0001 at m=2 (9 evaluations)


### Problem 1.3
*Using the Gauss-Legendre quadrature, which is a Gaussian quadrature with the weight function $w(x) = 1$ on $[0, 1]$, at least how many numbers of points do you need to achieve the absolute error smaller than $10^{-8}$ for $f_1$? The same question for $f_2$ to achieve the absolute error smaller than $10^{-4}$.*

*In this sub-problem, you can use existing libraries for Gauss-Legendre quadrature such as `numpy.polynomial.legendre.leggaus`*

In [58]:
def gauss_legendre(f: Callable, m: int, a: int, b: int) -> float:
    x_list, weights = np.polynomial.legendre.leggauss(m)
    # Shift x-points to be in [a, b] instead of default [-1, 1]
    shifted_x = (
        0.5                 # Scale to [-0.5, 0.5]
            * (x_list + 1)  # Shift to [0, 1]
            * (b - a)         # Shift to [0, (b-a)]
        + a)                # Shift to [a, b]

    return sum(
        [weight * f(x) for weight, x in zip(weights, shifted_x)]
    ) * 0.5 * (b - a)       # Need to scale the quadrature formula too 

#### f1

In [63]:
treshold = 1e-8

m = 1
while True:
    gauss_legendre_estimate = gauss_legendre(f1, m, a=0, b=1)

    error = np.abs(integral_f1 - gauss_legendre_estimate)

    print(f"Error for m={m}: {error}")
    if error < treshold:
        print(f"Error passed treshold {treshold} at m={m}")
        break

    m += 1

Error for m=1: 0.02558989911592424
Error for m=2: 0.00014179929671220837
Error for m=3: 3.031644896234198e-07
Error for m=4: 3.432196749031391e-10
Error passed treshold 1e-08 at m=4


#### f2

In [64]:
treshold = 1e-4

m = 1
while True:
    gauss_legendre_estimate = gauss_legendre(f2, m, a=0, b=1)

    error = np.abs(integral_f2 - gauss_legendre_estimate)

    print(f"Error for m={m}: {error}")
    if error < treshold:
        print(f"Error passed treshold {treshold} at m={m}")
        break

    m += 1

Error for m=1: 0.21306131942526685
Error for m=2: 0.03768310748973558
Error for m=3: 0.03466605762534214
Error for m=4: 0.010587330175454657
Error for m=5: 0.013765995217472793
Error for m=6: 0.004964227020994083
Error for m=7: 0.007359555828410191
Error for m=8: 0.002878988260852644
Error for m=9: 0.004575384819653139
Error for m=10: 0.0018791520579409804
Error for m=11: 0.0031182244913130486
Error for m=12: 0.0013229740601868878
Error for m=13: 0.002260999010056941
Error for m=14: 0.000981838273164004
Error for m=15: 0.0017143175247024134
Error for m=16: 0.0007575586317827288
Error for m=17: 0.00134441854581413
Error for m=18: 0.0006022393961538919
Error for m=19: 0.0010825288445107661
Error for m=20: 0.0004902403106687991
Error for m=21: 0.0008903424186633702
Error for m=22: 0.0004068232526099891
Error for m=23: 0.0007451468096525415
Error for m=24: 0.00034302596590263246
Error for m=25: 0.0006327812051626358
Error for m=26: 0.0002931433933264227
Error for m=27: 0.000544044086723927